# HSE 2021: Mathematical Methods for Data Analysis



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.datasets import load_boston

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import sklearn.metrics as metrics
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
import numpy as np
from sklearn.model_selection import GridSearchCV

%matplotlib inline

sns.set(style="darkgrid")

### Data

For this homework we use Boston Dataset from sklearn (based on UCI ML housing dataset).

In [ ]:
data = load_boston() # load dataset

X = data.data
y = data.target
columns = data.feature_names


## Linear regression

#### 1. Create Pandas DataFrame and split the data into train and test sets with ratio 80:20 with random_state=0.

In [ ]:
datafr = pd.DataFrame(X, columns = columns)
datafr['target'] = y
X_train, X_test, y_train, y_test = train_test_split(datafr.drop('target', axis=1), datafr.target, 
                                                    test_size=0.2, random_state=0)
datafr.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 2.  Train models on train data using StatsModels library and apply it to the test set; use $RMSE$ and $R^2$ as the quality measure.

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.01$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.01$

Don't forget to scale the data before training the models with StandardScaler!

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns = X_train.columns)

X_train_scaled_df = sm.add_constant(X_train_scaled_df)
X_test_scaled = sm.add_constant(X_test_scaled)
X_train_scaled = sm.add_constant(X_train_scaled)

In [ ]:
y_train_rs=y_train.values.reshape(-1,1)

model = sm.OLS(y_train_rs, X_train_scaled_df)
results = model.fit()
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.765    
Dependent Variable: y                AIC:                2370.9385
Date:               2021-03-13 17:20 BIC:                2426.9583
No. Observations:   404              Log-Likelihood:     -1171.5  
Df Model:           13               F-statistic:        102.2    
Df Residuals:       390              Prob (F-statistic): 9.64e-117
R-squared:          0.773            Scale:              20.020   
--------------------------------------------------------------------
           Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const     22.6119     0.2226   101.5764   0.0000   22.1742   23.0495
CRIM      -0.9708     0.2980    -3.2575   0.0012   -1.5568   -0.3849
ZN         1.0571     0.3408     3.1022   0.0021    0.3872    1.7271
INDUS      0.0383     0.4428     0.0865   0.9311   -0.8324    0.9090
CHAS       0.5945     0.2291     2.5946   0.0098    0.1440    1.0450
NOX       -1.8551     0.4846    -3.8282   0.0002   -2.8079   -0.9024
RM         2.5732     0.3175     8.1058   0.0000    1.9491    3.1974
AGE       -0.0876     0.4022    -0.2178   0.8277   -0.8784    0.7032
DIS       -2.8809     0.4446    -6.4800   0.0000   -3.7550   -2.0068
RAD        2.1122     0.6069     3.4805   0.0006    0.9191    3.3054
TAX       -1.8753     0.6652    -2.8191   0.0051   -3.1832   -0.5675
PTRATIO   -2.2928     0.3003    -7.6359   0.0000   -2.8831   -1.7024
B          0.7182     0.2613     2.7486   0.0063    0.2045    1.2319
LSTAT     -3.5925     0.3954    -9.0855   0.0000   -4.3698   -2.8151
------------------------------------------------------------------
Omnibus:             141.494       Durbin-Watson:          1.996  
Prob(Omnibus):       0.000         Jarque-Bera (JB):       629.882
Skew:                1.470         Prob(JB):               0.000  
Kurtosis:            8.365         Condition No.:          10     
==================================================================

"""

In [ ]:
y_test_pred = results.predict(X_test_scaled)
y_train_pred = results.predict(X_train_scaled)

rmse_test = rmse(y_test, y_test_pred)
rmse_train = rmse(y_train, y_train_pred)
print("Test RMSE = ", rmse_test)
print("Train RMSE = ", rmse_train)

Test RMSE =  5.783509315085143
Train RMSE =  4.396188144698282


In [ ]:
results_ridge = model.fit_regularized(L1_wt=0, alpha=0.01, start_params=results.params)
final_ridge = sm.regression.linear_model.OLSResults(model, 
                                              results_ridge.params, 
                                              model.normalized_cov_params)

final_ridge.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.765    
Dependent Variable: y                AIC:                2372.3540
Date:               2021-03-13 17:20 BIC:                2428.3738
No. Observations:   404              Log-Likelihood:     -1172.2  
Df Model:           13               F-statistic:        101.7    
Df Residuals:       390              Prob (F-statistic): 1.90e-116
R-squared:          0.772            Scale:              20.091   
--------------------------------------------------------------------
           Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const     22.3880     0.2230   100.3946   0.0000   21.9496   22.8264
CRIM      -0.9389     0.2986    -3.1449   0.0018   -1.5259   -0.3519
ZN         0.9965     0.3414     2.9190   0.0037    0.3253    1.6676
INDUS     -0.0576     0.4436    -0.1298   0.8968   -0.9298    0.8146
CHAS       0.6098     0.2295     2.6566   0.0082    0.1585    1.0611
NOX       -1.7222     0.4854    -3.5477   0.0004   -2.6766   -0.7678
RM         2.6120     0.3180     8.2135   0.0000    1.9868    3.2372
AGE       -0.1155     0.4029    -0.2867   0.7745   -0.9078    0.6767
DIS       -2.7539     0.4454    -6.1834   0.0000   -3.6295   -1.8783
RAD        1.8348     0.6079     3.0181   0.0027    0.6396    3.0301
TAX       -1.6265     0.6664    -2.4407   0.0151   -2.9366   -0.3163
PTRATIO   -2.2561     0.3008    -7.5007   0.0000   -2.8475   -1.6648
B          0.7188     0.2617     2.7463   0.0063    0.2042    1.2334
LSTAT     -3.5287     0.3961    -8.9086   0.0000   -4.3074   -2.7499
------------------------------------------------------------------
Omnibus:             147.660       Durbin-Watson:          1.990  
Prob(Omnibus):       0.000         Jarque-Bera (JB):       694.793
Skew:                1.521         Prob(JB):               0.000  
Kurtosis:            8.659         Condition No.:          10     
==================================================================

"""

In [ ]:
y_test_pred = final_ridge.predict(X_test_scaled)
y_train_pred = final_ridge.predict(X_train_scaled)

rmse_test_ridge = rmse(y_test, y_test_pred)
rmse_train_ridge = rmse(y_train, y_train_pred)
print("Test RMSE = ", rmse_test_ridge)
print("Train RMSE = ", rmse_train_ridge)

Test RMSE =  5.827028146610334
Train RMSE =  4.403896239747625


In [ ]:
results_lasso = model.fit_regularized(L1_wt=1, alpha=0.01, start_params=results.params)
final_lasso = sm.regression.linear_model.OLSResults(model, 
                                              results_lasso.params, 
                                              model.normalized_cov_params)

final_lasso.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.765    
Dependent Variable: y                AIC:                2371.0437
Date:               2021-03-13 17:20 BIC:                2427.0635
No. Observations:   404              Log-Likelihood:     -1171.5  
Df Model:           13               F-statistic:        102.1    
Df Residuals:       390              Prob (F-statistic): 1.01e-116
R-squared:          0.773            Scale:              20.025   
--------------------------------------------------------------------
           Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const     22.6019     0.2226   101.5182   0.0000   22.1642   23.0396
CRIM      -0.9404     0.2981    -3.1549   0.0017   -1.5264   -0.3543
ZN         1.0215     0.3408     2.9973   0.0029    0.3515    1.6916
INDUS      0.0000     0.4429     0.0000   1.0000   -0.8708    0.8708
CHAS       0.5948     0.2292     2.5955   0.0098    0.1442    1.0453
NOX       -1.8030     0.4847    -3.7202   0.0002   -2.7559   -0.8502
RM         2.5851     0.3175     8.1423   0.0000    1.9609    3.2094
AGE       -0.0690     0.4023    -0.1715   0.8639   -0.8599    0.7220
DIS       -2.8085     0.4446    -6.3163   0.0000   -3.6828   -1.9343
RAD        1.9569     0.6070     3.2241   0.0014    0.7636    3.1502
TAX       -1.7393     0.6653    -2.6142   0.0093   -3.0473   -0.4312
PTRATIO   -2.2788     0.3003    -7.5885   0.0000   -2.8692   -1.6884
B          0.7056     0.2613     2.7000   0.0072    0.1918    1.2194
LSTAT     -3.5969     0.3955    -9.0956   0.0000   -4.3744   -2.8194
------------------------------------------------------------------
Omnibus:             143.583       Durbin-Watson:          1.997  
Prob(Omnibus):       0.000         Jarque-Bera (JB):       649.866
Skew:                1.488         Prob(JB):               0.000  
Kurtosis:            8.454         Condition No.:          10     
==================================================================

"""

In [ ]:
y_test_pred = final_lasso.predict(X_test_scaled)
y_train_pred = final_lasso.predict(X_train_scaled)

rmse_test_lasso = rmse(y_test, y_test_pred)
rmse_train_lasso = rmse(y_train, y_train_pred)
print("Test RMSE = ", rmse_test_lasso)
print("Train RMSE = ", rmse_train_lasso)

Test RMSE =  5.796199043424474
Train RMSE =  4.396760341407677


**As we can see, the best performance indicators of RMSE on train and test sets are in the first Linear regression model. Also it has one of the highest R^2 result, which means that this model best matches the data.**

#### 3. Explore the values of the parameters of the resulting models and compare the number of zero weights in them. Comment on the significance of the coefficients, overal model significance and other related factors from the results table

Below we can see the tables with all neccessary information about each model.
First of all let's pay attention to the values of the parameters. We are aware, that Ridge and Lasso regression help us to reduce the value of parameters. That's why we can see in Lasso model one zero weight parameter (INDUS). It means that proportion of non-retail business acres per town does not effect our target. In other models the value of that parameter is close to zero.

To determine the significance of the coefficient we need to take a look to the p-value results. 

I'd like to use significance level of alpha = 0.05. 
In all 3 type of models we can see, that p-value coefficient of (INDUS) and (AGE) is more than selected level of significance. That means that they can be removed from our model (they do not effect the target). But still sometimes after removing one parameter the other can become more significant.

Well, now we can look at the r^2 result of each model. This parameter can help us to tell how good our model descides the data as it close to 1. The highest R^2 is in Lasso and Linear Regression models. 

Also we can take a look to F-statistic results. This statistic is used for accessing the significance of all model. The null hypotesis is that our model is equal to model where target = const (in our case: 22.612 or 22.602 or 22.388). Given Prob (F-statistic):	9.64e-117 and Prob (F-statistic):	1.01e-116 and Prob (F-statistic):	1.90e-116 are close to zero and the f-statistics values are large, which means that our models are significant.



In [ ]:
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.765    
Dependent Variable: y                AIC:                2370.9385
Date:               2021-03-13 17:20 BIC:                2426.9583
No. Observations:   404              Log-Likelihood:     -1171.5  
Df Model:           13               F-statistic:        102.2    
Df Residuals:       390              Prob (F-statistic): 9.64e-117
R-squared:          0.773            Scale:              20.020   
--------------------------------------------------------------------
           Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const     22.6119     0.2226   101.5764   0.0000   22.1742   23.0495
CRIM      -0.9708     0.2980    -3.2575   0.0012   -1.5568   -0.3849
ZN         1.0571     0.3408     3.1022   0.0021    0.3872    1.7271
INDUS      0.0383     0.4428     0.0865   0.9311   -0.8324    0.9090
CHAS       0.5945     0.2291     2.5946   0.0098    0.1440    1.0450
NOX       -1.8551     0.4846    -3.8282   0.0002   -2.8079   -0.9024
RM         2.5732     0.3175     8.1058   0.0000    1.9491    3.1974
AGE       -0.0876     0.4022    -0.2178   0.8277   -0.8784    0.7032
DIS       -2.8809     0.4446    -6.4800   0.0000   -3.7550   -2.0068
RAD        2.1122     0.6069     3.4805   0.0006    0.9191    3.3054
TAX       -1.8753     0.6652    -2.8191   0.0051   -3.1832   -0.5675
PTRATIO   -2.2928     0.3003    -7.6359   0.0000   -2.8831   -1.7024
B          0.7182     0.2613     2.7486   0.0063    0.2045    1.2319
LSTAT     -3.5925     0.3954    -9.0855   0.0000   -4.3698   -2.8151
------------------------------------------------------------------
Omnibus:             141.494       Durbin-Watson:          1.996  
Prob(Omnibus):       0.000         Jarque-Bera (JB):       629.882
Skew:                1.470         Prob(JB):               0.000  
Kurtosis:            8.365         Condition No.:          10     
==================================================================

"""

In [ ]:
final_lasso.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.765    
Dependent Variable: y                AIC:                2371.0437
Date:               2021-03-13 17:20 BIC:                2427.0635
No. Observations:   404              Log-Likelihood:     -1171.5  
Df Model:           13               F-statistic:        102.1    
Df Residuals:       390              Prob (F-statistic): 1.01e-116
R-squared:          0.773            Scale:              20.025   
--------------------------------------------------------------------
           Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const     22.6019     0.2226   101.5182   0.0000   22.1642   23.0396
CRIM      -0.9404     0.2981    -3.1549   0.0017   -1.5264   -0.3543
ZN         1.0215     0.3408     2.9973   0.0029    0.3515    1.6916
INDUS      0.0000     0.4429     0.0000   1.0000   -0.8708    0.8708
CHAS       0.5948     0.2292     2.5955   0.0098    0.1442    1.0453
NOX       -1.8030     0.4847    -3.7202   0.0002   -2.7559   -0.8502
RM         2.5851     0.3175     8.1423   0.0000    1.9609    3.2094
AGE       -0.0690     0.4023    -0.1715   0.8639   -0.8599    0.7220
DIS       -2.8085     0.4446    -6.3163   0.0000   -3.6828   -1.9343
RAD        1.9569     0.6070     3.2241   0.0014    0.7636    3.1502
TAX       -1.7393     0.6653    -2.6142   0.0093   -3.0473   -0.4312
PTRATIO   -2.2788     0.3003    -7.5885   0.0000   -2.8692   -1.6884
B          0.7056     0.2613     2.7000   0.0072    0.1918    1.2194
LSTAT     -3.5969     0.3955    -9.0956   0.0000   -4.3744   -2.8194
------------------------------------------------------------------
Omnibus:             143.583       Durbin-Watson:          1.997  
Prob(Omnibus):       0.000         Jarque-Bera (JB):       649.866
Skew:                1.488         Prob(JB):               0.000  
Kurtosis:            8.454         Condition No.:          10     
==================================================================

"""

In [ ]:
final_ridge.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.765    
Dependent Variable: y                AIC:                2372.3540
Date:               2021-03-13 17:20 BIC:                2428.3738
No. Observations:   404              Log-Likelihood:     -1172.2  
Df Model:           13               F-statistic:        101.7    
Df Residuals:       390              Prob (F-statistic): 1.90e-116
R-squared:          0.772            Scale:              20.091   
--------------------------------------------------------------------
           Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const     22.3880     0.2230   100.3946   0.0000   21.9496   22.8264
CRIM      -0.9389     0.2986    -3.1449   0.0018   -1.5259   -0.3519
ZN         0.9965     0.3414     2.9190   0.0037    0.3253    1.6676
INDUS     -0.0576     0.4436    -0.1298   0.8968   -0.9298    0.8146
CHAS       0.6098     0.2295     2.6566   0.0082    0.1585    1.0611
NOX       -1.7222     0.4854    -3.5477   0.0004   -2.6766   -0.7678
RM         2.6120     0.3180     8.2135   0.0000    1.9868    3.2372
AGE       -0.1155     0.4029    -0.2867   0.7745   -0.9078    0.6767
DIS       -2.7539     0.4454    -6.1834   0.0000   -3.6295   -1.8783
RAD        1.8348     0.6079     3.0181   0.0027    0.6396    3.0301
TAX       -1.6265     0.6664    -2.4407   0.0151   -2.9366   -0.3163
PTRATIO   -2.2561     0.3008    -7.5007   0.0000   -2.8475   -1.6648
B          0.7188     0.2617     2.7463   0.0063    0.2042    1.2334
LSTAT     -3.5287     0.3961    -8.9086   0.0000   -4.3074   -2.7499
------------------------------------------------------------------
Omnibus:             147.660       Durbin-Watson:          1.990  
Prob(Omnibus):       0.000         Jarque-Bera (JB):       694.793
Skew:                1.521         Prob(JB):               0.000  
Kurtosis:            8.659         Condition No.:          10     
==================================================================

"""

#### 4. Implement one of the elimination algorithms that were described in the Seminar_4 (Elimination by P-value, Forward elimination, Backward elimination), make conclusions.

In [ ]:
def Elimination(X_train, y, level, columns):
    best_columns = columns
    best_X = X_train
    reg_OLS = sm.OLS(y, X_train).fit()
    best_R_value = reg_OLS.rsquared_adj
    best_OLS = sm.OLS(y, X_train).fit()
    num_col = len(X_train[0])
    for i in range(0, num_col):
        reg_OLS = sm.OLS(y, X_train).fit()
        maxPval = max(reg_OLS.pvalues)
        if maxPval >= level:
            for j in range(0, num_col - i):
                if (reg_OLS.pvalues[j] == maxPval):
                    columns = np.delete(columns, j-1)
                    X_train = np.delete(X_train, j, 1)
                    if (reg_OLS.rsquared_adj > best_R_value):
                      best_OLS = sm.OLS(y, X_train).fit()
                      best_R_value = reg_OLS.rsquared_adj
                      best_columns = columns
                      best_X = X_train
    print(best_OLS.summary())
    return best_X, best_columns


In [ ]:
extra_columns = columns
new_model, best_columns = Elimination(X_train_scaled, y_train, 0.05, extra_columns)
print (best_columns)

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     121.3
Date:                Sat, 13 Mar 2021   Prob (F-statistic):          8.15e-119
Time:                        17:20:23   Log-Likelihood:                -1171.5
No. Observations:                 404   AIC:                             2367.
Df Residuals:                     392   BIC:                             2415.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.6119      0.222    101.829      0.0

**So as we can see the best result (in terms of R^2) we can get when we throw away INDUS and AGE.**

#### 5. Find the best (in terms of RMSE) $\alpha$ for Ridge regression using cross-validation with 5 folds. You must select values from range $[10^{-4}, 10^{3}]$.

In [ ]:
alpha = np.linspace(10**(-4), 10**3, 1000)
find_alpha = GridSearchCV(Ridge(), [{"alpha": alpha}], scoring="neg_root_mean_squared_error", cv=5)
find_alpha.fit(X_train_scaled, y_train)
best_res = find_alpha.best_params_["alpha"]
print("Best alpha = %.4f" % best_res)

Best alpha = 8.0081


In [ ]:
# Let's check

results_ridge = model.fit_regularized(L1_wt=0, alpha=0.01, start_params=results.params)
final_ridge = sm.regression.linear_model.OLSResults(model, 
                                              results_ridge.params, 
                                              model.normalized_cov_params)

y_test_pred = final_ridge.predict(X_test_scaled)
y_train_pred = final_ridge.predict(X_train_scaled)

rmse_test_ridge = rmse(y_test, y_test_pred)
rmse_train_ridge = rmse(y_train, y_train_pred)
print("Test RMSE = ", rmse_test_ridge)
print("Train RMSE = ", rmse_train_ridge)

Test RMSE =  5.827028146610334
Train RMSE =  4.403896239747625


In [ ]:
results_ridge = model.fit_regularized(L1_wt=0, alpha=8.0081, start_params=results.params)
final_ridge = sm.regression.linear_model.OLSResults(model, 
                                              results_ridge.params, 
                                              model.normalized_cov_params)

y_test_pred = final_ridge.predict(X_test_scaled)
y_train_pred = final_ridge.predict(X_train_scaled)

rmse_test_ridge = rmse(y_test, y_test_pred)
rmse_train_ridge = rmse(y_train, y_train_pred)
print("Test RMSE = ", rmse_test_ridge)
print("Train RMSE = ", rmse_train_ridge)

Test RMSE =  21.37501050517079
Train RMSE =  21.38700573335917


In [ ]:
results_ridge = model.fit_regularized(L1_wt=0, alpha=10.0081, start_params=results.params)
final_ridge = sm.regression.linear_model.OLSResults(model, 
                                              results_ridge.params, 
                                              model.normalized_cov_params)

y_test_pred = final_ridge.predict(X_test_scaled)
y_train_pred = final_ridge.predict(X_train_scaled)

rmse_test_ridge = rmse(y_test, y_test_pred)
rmse_train_ridge = rmse(y_train, y_train_pred)
print("Test RMSE = ", rmse_test_ridge)
print("Train RMSE = ", rmse_train_ridge)

Test RMSE =  21.824149608598937
Train RMSE =  21.895620668368803
